In [3]:
import  numpy as  np
import  pandas as  pd
import re
import itertools
import copy
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
import collections
from collections import defaultdict
mystem = Mystem()
morph = MorphAnalyzer()

## the Levenshtein distance

In [4]:
data = pd.read_csv('fon_table.csv/data.csv')
data = data.set_index('name')
data = data.transpose()
data = data.fillna('None')
data = pd.DataFrame(data)
data.head()

name,и,ы,е,э,ä,а,у,у_1,о,ъ,...,р’,j,к,к’,г,г’,х,х’,ɣ,ŋ
son,+,+,+,+,+,+,+,+,+,+,...,+,+,-,-,-,-,-,-,-,+
voc,+,+,+,+,+,+,+,+,+,+,...,+,-,-,-,-,-,-,-,-,-
cons,-,-,-,-,-,-,-,-,-,-,...,+,+,+,+,+,+,+,+,+,+
cor,None,None,None,None,None,None,None,None,None,None,...,+,-,-,-,-,-,-,-,-,-
ant,None,None,None,None,None,None,None,None,None,None,...,-,-,-,-,-,-,-,-,-,-


In [ ]:
data.to_csv('data.csv')

In [5]:
dist_matrix = pd.DataFrame(np.zeros((54, 54)))
column_names = {}

for i in range(0, 54):
    column_names[i] = data.columns.tolist()[i]
    
dist_matrix = dist_matrix.rename(index=str, columns=column_names)
dist_matrix['index'] = data.columns.tolist()
dist_matrix = dist_matrix.set_index('index')

Заполняю клеточки в таблице результатами формулы

1 - S_rows/ C_rows + Unc_rows*2

In [6]:
comobinations = itertools.combinations_with_replacement(data.columns, 2) # - все возможные комбинации

In [7]:
for i in comobinations:

    a1 = pd.get_dummies(data[i[0]]).drop(['None'], axis=1)
    a = a1.loc[~(a1==0).all(axis=1)]
    b1 = pd.get_dummies(data[i[1]]).drop(['None'], axis=1)
    b = b1.loc[~(b1==0).all(axis=1)]
    
    dif_a = a.index.difference(b.index)
    similar = a.index.difference(dif_a)
    
    d = a.loc[similar] + b.loc[similar]
    s_rows = 0
    counts = d['+'].value_counts()

    if 0 in counts:
        s_rows += counts[0]
    if 2 in counts:
        s_rows += counts[2]
    
                         # common                uncommon
    result = 1 - s_rows/(len(similar) + 2*(a.shape[0] + b.shape[0] - 2*len(similar)))

    dist_matrix[i[0]][i[1]] = result
    dist_matrix[i[1]][i[0]] = result

In [162]:
dist_matrix#.head()

,и,ы,е,э,ä,а,у,у_1,о,ъ,...,р’,j,к,к’,г,г’,х,х’,ɣ,ŋ
index,,,,,,,,,,,,,,,,,,,,,
и,0.000000,0.125000,0.125000,0.250000,0.250000,0.375000,0.250000,0.125000,0.375000,0.375000,...,0.875000,0.909091,1.000000,0.961538,1.000000,0.961538,1.000000,0.958333,1.000000,0.958333
ы,0.125000,0.000000,0.250000,0.125000,0.375000,0.250000,0.125000,0.250000,0.250000,0.250000,...,0.916667,0.954545,0.961538,1.000000,0.961538,1.000000,0.958333,1.000000,0.958333,0.916667
е,0.125000,0.250000,0.000000,0.125000,0.125000,0.250000,0.375000,0.250000,0.250000,0.250000,...,0.875000,0.909091,1.000000,0.961538,1.000000,0.961538,1.000000,0.958333,1.000000,0.958333
э,0.250000,0.125000,0.125000,0.000000,0.250000,0.125000,0.250000,0.375000,0.125000,0.125000,...,0.916667,0.954545,0.961538,1.000000,0.961538,1.000000,0.958333,1.000000,0.958333,0.916667
ä,0.250000,0.375000,0.125000,0.250000,0.000000,0.125000,0.500000,0.375000,0.375000,0.375000,...,0.875000,0.909091,1.000000,0.961538,1.000000,0.961538,1.000000,0.958333,1.000000,0.958333
а,0.375000,0.250000,0.250000,0.125000,0.125000,0.000000,0.375000,0.500000,0.250000,0.250000,...,0.916667,0.954545,0.961538,1.000000,0.961538,1.000000,0.958333,1.000000,0.958333,0.916667
у,0.250000,0.125000,0.375000,0.250000,0.500000,0.375000,0.000000,0.125000,0.125000,0.375000,...,0.916667,0.954545,0.961538,1.000000,0.961538,1.000000,0.958333,1.000000,0.958333,0.916667
у_1,0.125000,0.250000,0.250000,0.375000,0.375000,0.500000,0.125000,0.000000,0.250000,0.500000,...,0.875000,0.909091,1.000000,0.961538,1.000000,0.961538,1.000000,0.958333,1.000000,0.958333
о,0.375000,0.250000,0.250000,0.125000,0.375000,0.250000,0.125000,0.250000,0.000000,0.250000,...,0.916667,0.954545,0.961538,1.000000,0.961538,1.000000,0.958333,1.000000,0.958333,0.916667


In [ ]:
dist_matrix.to_csv('distance_matrix.csv')

In [ ]:
f = open('ru_st_normalized.csv', 'a', encoding='utf-8')
for index, row in st_words.iterrows():
    f.write(str(index) + ';' + str(row['stressed_s']) + '\n')
f.close()

In [ ]:
f = open('distance_matrix.csv', 'w', encoding='uft-8')
for index, row in dist_matrix.iterrows():
    f.write(str(index) + ';' + str(row['stressed_s']) + '\n')

## Расстояние Левенштейна

In [9]:
def lev_distance(a, b):
    
    # Первыми - строчки 
    # столбики - слово b
    
    dis = np.zeros((len(a) + 1, len(b) + 1))
    i = 0
    row = 0
    col = 0

    while i < dis.size:

        if row == 0:
            if col != 0:
                dis[row, col] = dis[row, col-1] + 1

        elif col == 0:
            if row != 0:
                dis[row, col] = dis[row - 1, col] + 1
        
  
        elif row > 1 and col > 1 and a[row-1] == b[col-2] and a[row-2] == b[col-1]:
            dis[row, col] = dis[row - 3][col - 3] + 1
            
        
        else:
            dis[row, col] = np.min([dis[row, col - 1] + 1,  # левый 
                                    dis[row - 1, col - 1] + dist_matrix[a[row-1]][b[col-1]], # диаг               
                                    dis[row - 1, col] + 1]) # верхний
        
        col += 1
        i += 1  
        if col == len(b) + 1:
            col = 0
            row += 1
  
    return dis[len(a), len(b)]

## Фонетические изменения

ГЛАСНЫЕ

In [10]:
vows = pd.read_csv('vowels.csv', sep=';')
vows = vows.set_index('Unnamed: 0')
vows = vows.reindex(vows.index.rename('name'))
vows

,#,v2_hard,v2_soft,v1_hard,v1_sh,v1_soft,V,vn_hard,vn_soft
name,,,,,,,,,
а,а,ъ,ь,а,a,и,а,ъ,ь
я,а,ъ,ь,а,a,и,а,ъ,ь
о,а,ъ,ь,а,ы,и,о,ъ,ь
ё,а,ъ,ь,а,ы,и,о,ъ,ь
у,у,у,у,у,у,у,у,у,у
ю,у,у,у,у,у,у,у,у,у
е,и,ъ,ь,ы,ы,и,е,ъ,ь
э,и,ъ,ь,ы,ы,,э,ъ,ь
и,и,ъ,ь,ы,ы,и,и,ъ,ь


In [ ]:
vows.to_csv('vows.csv')

СОГЛАСНЫЕ

In [11]:
cons = pd.read_csv('cons.csv', sep=';')
cons = cons.set_index('Unnamed: 0')
cons = cons.reindex(cons.index.rename('name'))
cons

,soft,hard,voiced,no_voice
name,,,,
б,б’,б,б,п
в,в’,в,в,ф
г,г’,г,г,к
д,д’,д,д,т
ж,ж,ж,ж,ш
з,з’,з,з,с
й,j,j,j,j
к,к’,к,г,к
л,л’,л,л,л


In [ ]:
cons.to_csv('cons_.csv')

Таблица, которая помогает учесть ассимиляцию

In [72]:
aff = pd.read_csv('aff.csv', sep=';')
aff = aff.set_index('Unnamed: 0')
aff = aff.reindex(aff.index.rename('name'))
aff

,с,ш,щ,ч,ж
name,,,,,
с,-,ш,ш’,ш’,ж
сс,-,-,-,ш’,-
ст,-,-,-,ш’,-
з,-,ш,ш’,ш’,ж’
зд,-,-,ш’,ш’,-
т,ц,ч’,-,ч’,-
ть,ц,-,-,-,-
д,ц,ч’,-,-,-
ж,-,-,-,ш’,ж’


In [ ]:
aff.to_csv('aff_.csv')

### ТАБЛИЦА УДАРЕНИЙ

In [13]:
st_words = pd.read_csv('ru_reversed.csv', sep='\t')
new = st_words['word'].str.replace(r'[^\w\s]', '')
new = new.str.lower()
st_words['word'] =  new
st_words = st_words.drop_duplicates()
st_words = st_words.set_index('word')
st_words.head()

,stressed_s
word,
а,1
аа,2
ааа,3
аааа,4
ааааа,5


In [ ]:
st_words.to_csv('stress_data.csv')

### ТРАНСКРИПЦИЯ

Кол-во гласных в слова

In [14]:
def num_of_vowls(word):
    
    num = len(re.findall('(а|е|ё|о|и|я|ю|у|ы|э)', word))   

    if num:
        return num  
    else:
        return 1

In [14]:
num_of_vowls('мама')

2

In [15]:
num_of_vowls('vfa')

1

Функция, которая ищет заданное слово в словаре с ударениями. Возращает номер гласной, на которой стоит ударение. Чтобы не использовать дополнительную память, сделала из него генератор. Если слова в словаре нет, то предлагаются все возможные варинты

In [15]:
def stressed(word, st_words):
    
    try:
        a = st_words.loc[word]['stressed_s'].tolist()
   
        if isinstance(a, list):
            yield from a
        else:
            yield from [a]
    
    except KeyError:
        yield from range(1, num_of_vowls(word) + 1)

In [16]:
[i for i in stressed('замок', st_words)]

[2, 1]

In [17]:
for i in stressed('замок', st_words):
    print(i)

2
1


Изменения гласных

In [579]:
def j2(letter, index, length, vows, next_, position, j_):

    ans = vows.loc[letter][position]

    if letter in ('ю', 'е', 'ё', 'я'):
        
        if index == length - 1:
            j_ = True

        elif next_ in vows.index:
            j_ = True
            if 'vn' in position:
                ans = vows.loc[letter]['vn_hard']

        elif next_ in ('ь', 'ъ'):
            j_ = True
            if 'vn' in position:
                ans = vows.loc[letter]['vn_hard']

    elif letter in ('и', 'о') and next_ == 'ь':
        j_ = True
        if 'vn' in position:
                ans = vows.loc[letter]['vn_hard']
    
    return ans, j_

In [544]:
def vows_after_aff(ans2, index, answer):
    
    if data[ans2]['back'] == '+' and index > 0:
        if answer[-1] == 'е':
            answer[-1] = 'э'
        if answer[-1] == 'и':
            answer[-1] = 'ы'
        if answer[-1] == 'ь':
            answer[-1] = 'ъ'

    elif data[ans2]['back'] == '-' and index > 0:
        if answer[-1] == 'э':
            answer[-1] = 'е'
        if answer[-1] == 'ы':
            answer[-1] = 'и'
        if answer[-1] == 'ъ':
            answer[-1] = 'ь'
 

In [581]:
def cons_tranformer2(letter, no_voice, soft, voice, index, length, letters, gap, answer, vcd=False):

    if gap > 0:  # пропуск в случае ассимиляции
        gap -= 1
        return '', no_voice, soft, voice, gap
    
    if index + 1 <= length-1:
        next_ = letters[index+1]
    else:
        next_ = ''

    if letter in ('ч', 'ш', 'щ', 'ж'):
        ans = cons.loc[letter]['hard']
        soft = False
        if letter in ('ч', 'щ'):
            no_voice = False
            voice = False
    else:
        ans = letter

    if no_voice and ans:  # оглушение
        ans = cons.loc[ans]['no_voice']

    elif voice:  # озвончение
        ans = cons.loc[ans]['voiced']

    if vcd is True:
        if index == 0 or index-1 == 0 and next_ in ('ь', 'ъ'):
            ans = cons.loc[ans]['voiced']

    if not vcd:
        if index == 0 or index-1 == 0 and next_ in ('ь', 'ъ'):  # конец слова
            no_voice = True
            if data[ans]['vcd'] == '+':
                ans = cons.loc[ans]['no_voice']

    if ans in data.columns and data[ans]['son'] == '-' and ans not in ('в', "в’") and data[ans]['vcd'] == '+':  # озвончение слудующих
        voice = True
    
    elif ans in data.columns and data[ans]['vcd'] == '-' and index + 1 <= length-1:  # оглушение следующих

        if next_ not in ('ь', 'ъ') and next_ in data.columns and data[next_]['son'] == '-':
            no_voice = True
    
        elif index + 2 <= length-1 and letters[index+2] in data.columns and data[letters[index+2]]['son'] == '-':
            no_voice = True

    if index-1 >= 0:

        if len(answer) > index-1 and answer[index-1] == 'к' and letter == 'г':
            ans = 'х'
        
        elif len(answer) > index-1 and answer[index-1] == 'т' and ans == "ч’":
            ans = 'ш'

        elif len(answer) > index-1 and answer[index-1] in (ans, ans + "’"):
            return '', no_voice, soft, voice, gap

    if soft:  # смягчение
        ans = cons.loc[ans]['soft']

    soft = False
    return ans, no_voice, soft, voice, gap

In [582]:
def transcription2(word, stress=False, next_word=False, separate=True, stop=False, vcd=False):
    
    """
    Фунция, которая определяет фонетическую транскриацию для слова с заданным ударением,
    расчитываемым с конца слова. 
    
    Функция принимает 3 аргемента: 
    i - type int, номер гласного, на который падает ударение (отсчет производится с конца слова).
    word - type str, слово, для которого должен произвожится разбор. 
    
    >>> transcription(2, 'мама')
    ['м', 'а', 'м', 'ъ']
    
    >>> transcription(1, 'мама')
    ['м', 'а', 'м', 'а']
    
    
    Функция работает только с кириллическими символами. Если в слове содержатся не кириллические символы, вызывается ошибка.
    
    >>> transcription(2, 'papa')
    Traceback (most recent call last):
        ...
    ValueError: No vowles or not Cyrillic elements
    
    
    Если пользователь указывает номер гласной меньший кол-ва гласных, вызывается ошибка.
    
    >>> transcription(3, 'мама')
    Traceback (most recent call last):
        ...
    ValueError: There are only 2 vowel(s)
    """
    
    if not isinstance(word, str):
        raise ValueError('Wrong data type')
        
    if not isinstance(stress, int):
        raise ValueError('Wrong data type')
        
    if ' ' in word:
        raise ValueError('Enter a word, not a phrase')
        
    nums = num_of_vowls(word)

    if stress:
        if nums < stress or stress <= 0:
            raise ValueError('There are only {} vowel(s)'.format(str(num_of_vowls(word))))
        
    if not stress:
        stress = nums//2 + 1
    
    if stop == True:
        stress = -100000

    letters = list(change(word))[::-1]
    answer = []
    length = len(letters)
    vow_n = 0
    soft = False
    voice = False
    no_voice = False
    gap = False
    j_ = False
    special = False
        
    for index, letter in enumerate(letters):

        letter = letter.lower()
  
        if letter in vows.index:
        
            voice = False
            no_voice = False
            gap = 0
            soft = False
            vow_n += 1
        
            ans, soft, j_ = due_to_vow_table2(vow_n, stress, letter, index, length, letters, soft, j_)
            
            if j_ is True and index == 0 and vow_n != stress:
                special = True
    
        elif letter == 'ь' or letter == 'ъ': 

            if letter == 'ь':
                soft = True
                
            if letter == 'ъ':
                soft = False
            continue
    
        elif letter in cons.index:
        
            if letter == 'й':
                ans = 'ṷ'
            
            else:
                ans, no_voice, soft, voice, gap = cons_tranformer2(letter, no_voice, soft, voice, index, length, letters, gap, answer, vcd=vcd)
        
                if ans == '':
                    continue
        else:

            raise ValueError('Not Cyrillic')
        
        answer.append(ans)

        if j_:
            if special:
                answer.append('ṷ')
                special = False
            else:
                answer.append('j')
            j_ = False
            

    if separate == False:
        return ''.join(answer[::-1])
    return answer[::-1]

НОВЫЙ КОД

In [517]:
def change(word):
    word = re.sub('(с|ст|сс|з|зд|ж|ш)ч', 'щ', word)
    word = re.sub('(с|зд|з)щ', 'щ', word)
    word = re.sub('(тч|тш|дш)', 'ч', word)
    word = re.sub('(с|з)ш', 'ш', word)
    word = re.sub('сж', 'ж', word)
    word = re.sub('(т|ть|д)с', 'ц', word)
    word = re.sub('(ст|сть)с', 'ц', word)
    return word

In [406]:
def my_type(letter):
    
    if letter in vows.index:
        return 'v'
    
    if letter in cons.index:
        return 'c'
    
    if letter in ('ь', 'ъ'):
        return 'm'
    
    else:
        raise ValueError('Not Cyrillic')
    

In [ ]:
def vows_after_aff(ans2, index, answer):
    
    if data[ans2]['back'] == '+' and index > 0:
        if answer[-1] == 'е':
            answer[-1] = 'э'
        if answer[-1] == 'и':
            answer[-1] = 'ы'
        if answer[-1] == 'ь':
            answer[-1] = 'ъ'

    elif data[ans2]['back'] == '-' and index > 0:
        if answer[-1] == 'э':
            answer[-1] = 'е'
        if answer[-1] == 'ы':
            answer[-1] = 'и'
        if answer[-1] == 'ъ':
            answer[-1] = 'ь'
 

In [548]:
def due_to_vow_table(ans, index, letter, stress, vow_n, length):
    
    if vow_n == stress: # ударный 
     
        if ans.value not in ('а', 'о', 'у', 'ы') and ans.next is not None and ans.next.type == 'c':
            
            if ans.value != 'э' and ans.next.value not in ('ш', 'ж', 'ц'):
                ans.next.soft = True
            elif ans.value in ('э', 'е') and ans.next.value in ('ш', 'ж', 'ц'):
                letter = 'э'
            elif ans.value == 'и':
                letter = 'ы'
                
        j(ans, letter, length, index, 'V')
        
    elif index == length - 1:  # начало
        j(ans, letter, length, index, '#')
        
        
    elif vow_n == stress + 1: # первый предударный

        if ans.next is not None and ans.next.value in ('ц', 'ж', 'ш'):
            ans.value = vows.loc[letter]['v1_sh']
        
        elif letter in ('е', 'ё', 'и', 'ю', 'я'):
            ans.value = vows.loc[letter]['v1_soft']
            ans.next.soft = True
        else:
            ans.value = vows.loc[letter]['v1_hard']
            
            
    elif vow_n >= stress + 2: # второй предударный
        
        if ans.next is not None and ans.next.value in ('ц', 'ж', 'ш'):
            ans.value = vows.loc[letter]['v2_hard']
        
        elif ans.next is not None and ans.next.type == 'v':
            if letter in ('е', 'ё', 'и', 'ю', 'я'):
                ans.value = vows.loc[letter]['v1_soft']      
            else:
                ans.value = vows.loc[letter]['v1_hard']
                
        elif letter in ('е', 'ё', 'и', 'ю', 'я'):
            ans.value = vows.loc[letter]['v2_soft']
            ans.next.soft = True
        else:
            ans.value = vows.loc[letter]['v2_hard']
            
            
    elif vow_n < stress: # заударные 
        
        if ans.next is not None and ans.next.value in ('ц', 'ж', 'ш'):
   
            j(ans, letter, length, index, 'vn_hard')
   
        elif letter in ('е', 'ё', 'и', 'ю', 'я'):
            
            if vow_n == stress - 1 and ans.next is not None and ans.next.type == 'v':
                j(ans, letter, length, index, 'vn_soft')
            else:
                ans.value = vows.loc[letter]['vn_soft']
            ans.next.soft = True

        else:
            if vow_n == stress - 1 and ans.next is not None and ans.next.type == 'v':
                j(ans, letter, length, index, 'vn_hard')
            else:
                ans.value = vows.loc[letter]['vn_hard']   


In [549]:
def j(ans, letter, length, index, position):

    ans.value = vows.loc[letter][position]

    if letter in ('ю', 'е', 'ё', 'я'):
        
        if index == length - 1:
            ans.j_ = True

        elif ans.next is not None and ans.next.type == 'v':
            ans.j_ = True
     
            if 'vn' in position:
                ans.value = vows.loc[letter]['vn_hard']

        elif ans.next is not None and ans.next.value in ('ь', 'ъ'):
            ans.j_ = True

            if 'vn' in position:
                ans.value = vows.loc[letter]['vn_hard']

    elif letter in ('и', 'о') and ans.next is not None and ans.next.value == 'ь':
        ans.j_ = True

        if 'vn' in position:
                ans.value = vows.loc[letter]['vn_hard']
    


In [616]:
def cons_tranformer(ans, letter, vcd=False):
    
    a = letter

    if letter in ('ч', 'ш', 'щ', 'ж'):
        ans.value = cons.loc[ans.value]['hard']
        if ans.__dict__.__contains__('soft'):
            del ans.soft
        if letter in ('ч', 'щ'):
            if ans.__dict__.__contains__('no_voice'):
                del ans.no_voice
            if ans.__dict__.__contains__('voice'):
                del ans.voice

    if ans.__dict__.__contains__('no_voice'):  # оглушение
        ans.value = cons.loc[ans.value]['no_voice']

    elif ans.__dict__.__contains__('voice'):  # озвончение
        ans.value = cons.loc[ans.value]['voiced']

    if vcd is True:
        if ans.previous is None:
            ans.value = cons.loc[ans.value]['voiced']

    if not vcd:
        if ans.previous is None:  # конец слова
            if ans.next is not None:
                ans.next.no_voice = True
            if data[ans.value]['vcd'] == '+':
                ans.value = cons.loc[ans.value]['no_voice']
    
    if ans.next is not None:
        if ans.value in data.columns and data[ans.value]['vcd'] == '-' and ans.next is not None:  # оглушение следующих

            if ans.next.value in data.columns and data[ans.next.value]['son'] == '-':
                ans.next.no_voice = True
    
        elif ans.value in data.columns and data[ans.value]['son'] == '-' and ans.value not in ('в', "в’") and data[ans.value]['vcd'] == '+':  # озвончение слудующих
            ans.next.voice = True

    if ans.previous is not None:

        if ans.previous.value == 'к' and letter == 'г':
            ans.value = 'х'
        
        elif ans.previous.value == 'т' and ans.value == "ч’":
            ans.value = 'ш'

        elif ans.previous.value in (ans.value, ans.value + "’"):
            ans.value = ''

    if ans.__dict__.__contains__('soft'):  # смягчение
        ans.value = cons.loc[ans.value]['soft']


In [558]:
class Node(object):
    def __init__(self, value=''):
        self.value = value
        self.type = None
        self.previous = None
        self.next = None


In [601]:
def transcription(word, stress=False, next_word=False, separate=True, stop=False, vcd=False):
    """
    Фунция, которая определяет фонетическую транскриацию для слова с заданным ударением,
    расчитываемым с конца слова. 
    
    Функция принимает 3 аргемента: 
    i - type int, номер гласного, на который падает ударение (отсчет производится с конца слова).
    word - type str, слово, для которого должен произвожится разбор. 
    
    >>> transcription(2, 'мама')
    ['м', 'а', 'м', 'ъ']
    
    >>> transcription(1, 'мама')
    ['м', 'а', 'м', 'а']
    
    
    Функция работает только с кириллическими символами. Если в слове содержатся не кириллические символы, вызывается ошибка.
    
    >>> transcription(2, 'papa')
    Traceback (most recent call last):
        ...
    ValueError: No vowles or not Cyrillic elements
    
    
    Если пользователь указывает номер гласной меньший кол-ва гласных, вызывается ошибка.
    
    >>> transcription(3, 'мама')
    Traceback (most recent call last):
        ...
    ValueError: There are only 2 vowel(s)
    """
   
    if not isinstance(word, str):
        raise ValueError('Wrong data type')
        
    if not isinstance(stress, int):
        raise ValueError('Wrong data type')
        
    if ' ' in word:
        raise ValueError('Enter a word, not a phrase')
        
    nums = num_of_vowls(word)

    if stress:
        if nums < stress or stress <= 0:
            raise ValueError('There are only {} vowel(s)'.format(str(num_of_vowls(word))))
        
    if not stress:
        stress = nums//2 + 1
    
    if stop == True:
        stress = -100000

    letters = list(change(word))[::-1]
    
    ans = Node()
    head = ans 
    prev = ans
    answer = []
    special = False
    
    length = len(letters)
    vow_n = 0
        
    for index, letter in enumerate(letters):

        letter = letter.lower()
        
        if index == 0:
            ans.value = letter
            ans.type = my_type(letter)
            head = ans
        else:
            ans.previous = prev
        
        if index != length - 1: # если не начало слова
            ans.next = Node(letters[index+1])
            ans.next.type = my_type(ans.next.value)
            
        if ans.type == 'v':  # гласные
            vow_n += 1
            due_to_vow_table(ans, index, letter, stress, vow_n, length)
            if ans.__dict__.__contains__('j_') and index == 0 and vow_n != stress:
                special = True
        
        elif ans.type == 'm':  # знаки

            if letter == 'ь':
                ans.next.soft = True
            ans.value = ''

        elif ans.type == 'c':  # согласные

            if letter == 'й':
                ans.value = 'ṷ'
            else:
                cons_tranformer(ans, letter)
        
        else:
            raise ValueError('Not Cyrillic')

        if ans.value == '' and index != 0:
            prev = ans.previous
        else:
            answer.append(ans.value)
            prev = ans
        
        if ans.__dict__.__contains__('j_'):
            if special:
                answer.append('ṷ')
                special = False
            else:
                answer.append('j')
        
        ans = ans.next
    
    if separate == False:
        return ''.join(answer[::-1])
    return answer[::-1]

In [603]:
import time
start_time = time.time()
print(transcription2('подтяжки', stress=2, separate=False))
print("--- %s seconds ---" % (time.time() - start_time))

пат’ашк’ь
--- 0.0024590492248535156 seconds ---


Результат новой версии

In [618]:
import time
start_time = time.time()
print(transcription('подтяжки', stress=2, separate=False))
print("--- %s seconds ---" % (time.time() - start_time))

пат’ашк’ь
--- 0.0017690658569335938 seconds ---


In [611]:
transcription('дт', stress=1, separate=False)

'т'

In [612]:
transcription('дрожжи', stress=2, separate=False)

'дрожъ'

In [29]:
stop_words = ['а',  'без',  'близ',  'в',  'вне',  'во',  'вокруг',  'вслед',  'для',  'до',  'за',  'и',  'из', 
              'изза',  'изо',  'изпод',  'к',  'ко',  'меж',  'между',  'мимо',  'на',  'над',  'о',  'об',  'обо', 
              'около',  'от',  'ото',  'перед',  'передо',  'по', 'поверх',  'под',  'подо',  'понад',  'после', 
              'пред',  'при',  'про',  'ради',  'с',  'сверх',  'сверху']

In [30]:
def tokenize(text):
    text = re.sub(r'[^\w\s]','',text).replace('\n', '')
    text = re.sub(r'[\s]{2,}',' ',text)
    return text.split(' ')

In [64]:
def phrase_transformer(text, separate=True):
    
    def combine(terms, accum):

        last = (len(terms) == 1)
        n = len(terms[0])

        for i in range(n):
            item = accum + [terms[0][i]]

            if last:
                combinations.append(item)
            else:
                combine(terms[1:], item)
                
    if not isinstance(text, str):
        raise ValueError('Wrong data type')

    num_words = 0
    words = tokenize(text)
    answer = []
    combinations = []
    length = len(words)

    for index, word in enumerate(words):

        stop = False
        vcd = False  # озвончение, если это фраза 
   
        if word in stop_words and length > 1:

            stop = True
            if index+1 <= len(words)-1 and words[index+1][0] in data.columns and data[words[index+1][0]]['vcd'] == '+':
                vcd = True
        
        answer.append([])
        
        for stress in stressed(word, st_words):
            num_words+=1
            if separate == False:
             
                answer[-1].append(''.join(transcription(word, stop=stop, vcd=vcd, stress=stress)))

            else:
                answer[-1].append(transcription(word, stop=stop, vcd=vcd, stress=stress))

    combine(answer, [])
    return combinations

In [47]:
a = phrase_transformer('замок')
for i in a:
    print(i)


[['з', 'а', 'м', 'ъ', 'к']]
[['з', 'а', 'м', 'о', 'к']]


In [65]:
phrase_transformer('котик')

[[['к', 'о', 'т’', 'ь', 'к']]]

In [78]:
phrase_transformer('воля', separate=False)

[['вол’ь']]

In [35]:
def phonetic_distance(word1, word2=False, transcription=False):
    
    word1 = phrase_transformer(word1)
        
    if len(word1[0]) > 1:
        raise ValueError('Enter values must be words, not phrases')
            
    word2 = phrase_transformer(word2)
        
    if len(word1[0]) > 1:
        raise ValueError('Enter values must be words, not phrases')

    answer = []
        
    for w1 in word1:
        for w2 in word2:

            if transcription:
                answer.append([''.join(w1[0]), ''.join(w2[0]), lev_distance(w1[0], w2[0])])
            else:
                answer.append(lev_distance(w1[0], w2[0]))
        
    return answer

In [219]:
transcription('мозг')

['м', 'а', 'с', 'к']

In [68]:
phonetic_distance('замок', 'заминка')

[3.0, 2.475]

In [ ]:
господи
ага
бога
бугалтер

In [ ]:
Добавить ситуацию со стоп словом, когда он 

Тестирую, если ли хорошо ставить ударение на 1 слог

In [119]:
import random
from sklearn.metrics import accuracy_score

In [133]:
a = random.sample(st_words.index.tolist(), 1500)

In [140]:
y_real = [] # реальное ударение
y_pred_first = [] # на первый слог
y_pred_last = []  # на последний слог
y_pred_random = [] # на рандомный
y_pred_mid = [] # на срединный 
y_pred_1 = []

for i in a:
    b = st_words.loc[i]['stressed_s']
    n = num_of_vowls(i)
    if not type(b) == pd.core.series.Series:
        y_real.append(b)
        y_pred_first.append(n)
        y_pred_last.append(1)
        y_pred_random.append(random.randint(1, n))
        y_pred_mid.append(n//2)
        y_pred_1.append(n//2 + 1)

In [135]:
accuracy_score(y_real, y_pred_first)

0.1108843537414966

In [136]:
accuracy_score(y_real, y_pred_last)

0.05510204081632653

In [137]:
accuracy_score(y_real, y_pred_random)

0.20612244897959184

In [138]:
accuracy_score(y_real, y_pred_mid)

0.24557823129251702

In [141]:
accuracy_score(y_real, y_pred_1)

0.42108843537414964

In [166]:
a = 'просчет'
re.sub('(сс|с|ст|з|зд)ч', 'щ', a)

'прощет'

In [ ]:
a = 'дрожжи'
re.sub('(сс|с|ст|з|зд)ж', 'щ', a)

In [ ]:
СТАРЫЕ